<a href="https://colab.research.google.com/github/pkonowrocki/Weather-prediction/blob/master/WindCodesComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#boilerplate code
import subprocess
file_id = '1XfiON89EFCsw5zhtD4hq1hn5Z21vGzpG'
subprocess.run(['pip', 'install', 'PyDrive'])
subprocess.run(['apt-get', 'install', 'unzip'])
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':file_id})
downloaded.GetContentFile('Dataset.zip')
if not os.path.exists('./Dataset'):
  subprocess.run(['unzip', './Dataset.zip'])
print(f'{"".join(["-" for _ in range(10)])}DATA READY{"".join(["-" for _ in range(10)])}')

#code
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from sklearn import preprocessing

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('cuda')
else:
    device = torch.device('cpu')
    print('cpu')

def testNetwork(Xtest, Ytest, net, criterion):
  x = torch.Tensor(Xtest).to(device)
  output = net(x)
  loss = criterion(output, torch.Tensor(np.array([Ytest]).T).to(device))
  return loss.item()/len(Ytest), output

def trainNetwork(path, code, net = None, epochs = None, criterion = None, checkEvery = None, optimizer = None, val = 0.0, scaler = None, batch = 1, gradClipping = 0, seed = 0):
  torch.manual_seed(seed)
  np.random.seed(seed)

  Xtrain = np.genfromtxt(f'{path}/{code}-train-input.csv', delimiter=',')
  Ytrain = np.genfromtxt(f'{path}/{code}-train-output.csv', delimiter=',')
  Xtest = np.genfromtxt(f'{path}/{code}-test-input.csv', delimiter=',')
  Ytest = np.genfromtxt(f'{path}/{code}-test-output.csv', delimiter=',')
  if scaler is None:
    scaler = preprocessing.StandardScaler().fit(Xtrain)
  Xtrain = scaler.transform(Xtrain)
  Xtest = scaler.transform(Xtest)

  if not val == 0.0:
    n = int(len(Ytrain)*val)+1 if int(len(Ytrain)*val)+1 < len(Ytrain) else int(len(Ytrain)*val)
    idxs = np.random.choice(np.arange(0, len(Ytrain), 1), n)
    Xval = Xtrain[idxs]
    Yval = Ytrain[idxs]
    Xtrain = np.delete(Xtrain, idxs, axis=0)
    Ytrain = np.delete(Ytrain, idxs)
  else:
    Xval = None

  if epochs is None:
    epochs = 1000
  if checkEvery is None:
    checkEvery = 5
  if net is None:
    numInputs = Xtrain.shape[1]
    numOutputs = 1
    hiddenSize = 50
    net = nn.Sequential(
      nn.Linear(numInputs, 2*hiddenSize),
      nn.ELU(),
      nn.Linear(2*hiddenSize, hiddenSize),
      nn.ELU(),
      nn.Linear(hiddenSize, numOutputs)).to(device)
  if criterion is None:
    criterion = nn.MSELoss(reduction='sum')
  if optimizer is None:
    optimizer = optim.Adadelta(net.parameters())

  net.to(device)

  if batch == 1:
    splitXtrain = Xtrain
    splitYtrain = Ytrain
  else:
    batch = int(len(Ytrain)/batch)
    splitXtrain = np.array_split(Xtrain, batch)
    splitYtrain = np.array_split(Ytrain, batch)

  for epoch in range(epochs):
    runningLoss = 0.0
    for i, x in enumerate(splitXtrain, 0):
      if np.any(np.isnan(x)):
        raise('NaN value in input')
      if np.any(np.isnan(splitYtrain[i])):
        raise('NaN value in target')
      optimizer.zero_grad()
      y = torch.Tensor(np.array([splitYtrain[i]]).T).to(device)
      x = torch.Tensor(x).to(device)
      output = net(x)
      if np.any(np.isnan(output.cpu().detach().numpy())):
        raise('NaN value in output')
      loss = criterion(output, y)
      loss.backward()
      if not gradClipping == 0:
        torch.nn.utils.clip_grad_norm_(net.parameters(), gradClipping)
      optimizer.step()
      runningLoss += loss.item()

    if epoch % checkEvery == checkEvery-1 or epoch == 0:
      if not Xval is None:
        lossVal, _ = testNetwork(Xval, Yval, net, criterion)
      else:  
        lossVal, _ = testNetwork(Xtest, Ytest, net, criterion)
      print(f'{epoch}, {runningLoss/len(Ytrain)}, {lossVal}')
  
  lossVal, _ = testNetwork(Xtest, Ytest, net, criterion)
  print(f'Finally: {lossVal}')



for _ in range(100):
  seed = np.random.randint(np.iinfo(np.int32).max)
  print(f'Clouds Okta Wind Gray (seed: {seed})')
  trainNetwork('./Dataset/Temperature/Imputation', 'CloudsOktaWindGray', val = 0.2, batch = 32, gradClipping = 2, seed = seed)
  print(f'Clouds Okta Wind Natural (seed: {seed})')
  trainNetwork('./Dataset/Temperature/Imputation', 'CloudsOktaWindNatural', val = 0.2, batch = 32, gradClipping = 2)

----------DATA READY----------
cuda
Clouds Okta Wind Gray (seed: 1259631606)
Sequential(
  (0): Linear(in_features=65, out_features=100, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ELU(alpha=1.0)
  (4): Linear(in_features=50, out_features=1, bias=True)
)
0, 32.229513796152304, 13.979010387859514
4, 5.023434774933289, 6.4963973053788715
9, 4.457913987462334, 6.026228373029591
14, 4.163417206331634, 5.46961421460177
19, 4.022195913301164, 5.207749628387721


KeyboardInterrupt: ignored